============================================
NOTEBOOK GOOGLE COLAB - TRADUTOR GPT-4O MINI
PRONTO PARA USAR - COPIE E COLE TUDO
============================================

In [ ]:
# ============================================
# CÉLULA 1: Instalar dependências
# ============================================
!pip install -q requests beautifulsoup4 python-docx openai python-dotenv
print("✅ Dependências instaladas com sucesso!")

✅ Dependências instaladas com sucesso!


In [ ]:
# ============================================
# CÉLULA 2: Importar bibliotecas
# ============================================
from google.colab import files
from openai import AzureOpenAI
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import os
from typing import Tuple, List
print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


============================================
CÉLULA 3: CLASSE COMPLETA - COPIE AQUI
============================================

In [ ]:
class TradutorGPT5Mini:
    """Tradutor profissional usando ChatGPT 5 Mini + Azure AI"""

    def __init__(self, api_key: str, api_version: str, azure_endpoint: str, deployment_name: str = "gpt-5-mini"):
        """Inicializa o cliente Azure OpenAI com GPT5 Mini"""
        self.client = AzureOpenAI(
            api_key=api_key,
            api_version=api_version,
            azure_endpoint=azure_endpoint
        )
        self.deployment_name = deployment_name

    def extrair_texto_url(self, url: str, max_caracteres: int = 50000) -> Tuple[str, str]:
        """Extrai texto de uma URL com tratamento robusto"""
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(url, headers=headers, timeout=15)
            response.raise_for_status()
            response.encoding = response.apparent_encoding

            soup = BeautifulSoup(response.content, 'html.parser')

            # Remover elementos desnecessários
            for element in soup(['script', 'style', 'nav', 'footer', 'button', 'form']):
                element.decompose()

            # Extrair título
            title = soup.find(['h1', 'title'])
            title_text = title.get_text(strip=True) if title else "Documento Traduzido"

            # Extrair conteúdo
            main_content = soup.find(['main', 'article']) or soup.find('body')
            if main_content:
                paragraphs = main_content.find_all(['p', 'h2', 'h3', 'li'])
            else:
                paragraphs = soup.find_all(['p', 'h2', 'h3', 'li'])

            content_parts = [p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)]
            content = '\n\n'.join(content_parts)

            if len(content) > max_caracteres:
                content = content[:max_caracteres] + "\n\n[... conteúdo truncado ...]"

            return title_text, content

        except Exception as e:
            raise Exception(f"❌ Erro ao extrair texto: {e}")

    def traduzir_texto(self, texto: str, idioma_origem: str = "português",
                      idioma_destino: str = "inglês") -> str:
        """Traduz texto usando ChatGPT 4o Mini"""
        try:
            chunks = self._dividir_em_chunks(texto, max_chars=8000)
            textos_traduzidos = []

            for i, chunk in enumerate(chunks, 1):
                if len(chunks) > 1:
                    print(f"  🔄 Traduzindo chunk {i}/{len(chunks)}...")

                prompt = f"""Traduza o seguinte texto de {idioma_origem} para {idioma_destino}.
Mantenha a formatação, estrutura de parágrafos e significado original.
Responda APENAS com o texto traduzido, sem explicações.

TEXTO:
{chunk}"""

                response = self.client.chat.completions.create(
                    model=self.deployment_name,
                    messages=[
                        {
                            "role": "system",
                            "content": f"Você é um tradutor profissional de {idioma_origem} para {idioma_destino}."
                        },
                        {
                            "role": "user",
                            "content": prompt
                        }
                    ],

                    max_completion_tokens=16384
                )

                texto_traduzido = response.choices[0].message.content.strip()
                textos_traduzidos.append(texto_traduzido)

            return '\n\n'.join(textos_traduzidos)

        except Exception as e:
            raise Exception(f"❌ Erro ao traduzir: {e}")

    def _dividir_em_chunks(self, texto: str, max_chars: int = 8000) -> List[str]:
        """Divide texto em chunks mantendo integridade"""
        paragrafos = texto.split('\n\n')
        chunks = []
        chunk_atual = []
        tamanho_atual = 0

        for paragrafo in paragrafos:
            tamanho_paragrafo = len(paragrafo)
            if tamanho_atual + tamanho_paragrafo > max_chars and chunk_atual:
                chunks.append('\n\n'.join(chunk_atual))
                chunk_atual = [paragrafo]
                tamanho_atual = tamanho_paragrafo
            else:
                chunk_atual.append(paragrafo)
                tamanho_atual += tamanho_paragrafo + 2

        if chunk_atual:
            chunks.append('\n\n'.join(chunk_atual))

        return chunks

    def salvar_em_docx(self, titulo: str, conteudo_original: str,
                      conteudo_traduzido: str, idioma_origem: str,
                      idioma_destino: str, caminho_saida: str, url: str = "") -> str:
        """Salva o conteúdo traduzido em arquivo DOCX formatado"""
        try:
            doc = Document()

            # Título
            heading = doc.add_heading(f"{titulo}", level=1)
            heading.alignment = WD_ALIGN_PARAGRAPH.CENTER
            heading.runs[0].font.color.rgb = RGBColor(0, 51, 102)

            # Informações
            info = doc.add_paragraph()
            info.add_run(f"Tradução: {idioma_origem.upper()} → {idioma_destino.upper()}").font.italic = True

            if url:
                doc.add_paragraph(f"Fonte: {url}").runs[0].font.size = Pt(9)

            doc.add_paragraph(f"Gerado em: {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}").runs[0].font.size = Pt(9)
            doc.add_paragraph()

            # Conteúdo traduzido
            doc.add_heading("📋 Conteúdo Traduzido", level=2)
            doc.paragraphs[-1].runs[0].font.color.rgb = RGBColor(0, 102, 51)

            for paragrafo in conteudo_traduzido.split('\n\n'):
                if paragrafo.strip():
                    p = doc.add_paragraph(paragrafo.strip())
                    p.paragraph_format.line_spacing = 1.15

            # Original (preview)
            doc.add_page_break()
            doc.add_heading("📄 Conteúdo Original", level=2)

            for paragrafo in conteudo_original.split('\n\n')[:3]:
                if paragrafo.strip():
                    doc.add_paragraph(paragrafo.strip())

            if len(conteudo_original.split('\n\n')) > 3:
                doc.add_paragraph("[... conteúdo truncado ...]").runs[0].font.italic = True

            doc.save(caminho_saida)
            return caminho_saida

        except Exception as e:
            raise Exception(f"❌ Erro ao salvar: {e}")

    def processar_url_completo(self, url: str, idioma_origem: str = "português",
                              idioma_destino: str = "inglês",
                              caminho_saida: str = "traducao.docx") -> str:
        """Processa URL completa: extrai → traduz → salva"""
        print(f"\n🚀 Iniciando tradução com ChatGPT 5 Mini")
        print(f"{'='*60}")

        print(f"\n🌐 Etapa 1: Extraindo texto...")
        titulo, conteudo_original = self.extrair_texto_url(url)
        print(f"   ✅ Título: {titulo}")
        print(f"   ✅ Caracteres: {len(conteudo_original):,}")

        print(f"\n🔄 Etapa 2: Traduzindo...")
        conteudo_traduzido = self.traduzir_texto(conteudo_original, idioma_origem, idioma_destino)
        print(f"   ✅ Tradução concluída")

        print(f"\n💾 Etapa 3: Salvando arquivo...")
        arquivo = self.salvar_em_docx(titulo, conteudo_original, conteudo_traduzido,
                                      idioma_origem, idioma_destino, caminho_saida, url)
        print(f"   ✅ {arquivo}")
        print(f"\n{'='*60}\n")

        return arquivo

        print("✅ Classe TradutorGPT5Mini carregada com sucesso!")

============================================
CÉLULA 4: Configurar credenciais Azure
============================================

In [ ]:
print("\n🔑 CONFIGURAÇÃO DA AZURE")
print("="*60)
print("\nOpções:")
print("1 - Inserir credenciais manualmente")
print("2 - Carregar arquivo .env")
opcao = input("\nEscolha (1 ou 2): ").strip()
if opcao == "2":
    print("\n📁 Carregando arquivo .env...")
    uploaded = files.upload()
    env_file = list(uploaded.keys())[0]

    from dotenv import load_dotenv
    load_dotenv(env_file)

    API_KEY = os.getenv('AZURE_API_KEY')
    AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
    DEPLOYMENT_NAME = os.getenv('DEPLOYMENT_NAME', 'gpt-5-mini')
    print("✅ Credenciais carregadas do .env")
else:
    print("\n📝 Digite suas credenciais:")
    API_KEY = input("   🔐 Azure API Key: ").strip()
    AZURE_ENDPOINT = input("   🌐 Azure Endpoint (ex: https://seu-recurso.openai.azure.com/): ").strip()
    DEPLOYMENT_NAME = input("   🤖 Deployment Name (padrão: gpt-5-mini): ").strip() or "gpt-5-mini"
    print("\n✅ Credenciais configuradas")


🔑 CONFIGURAÇÃO DA AZURE

Opções:
1 - Inserir credenciais manualmente
2 - Carregar arquivo .env

Escolha (1 ou 2): 2

📁 Carregando arquivo .env...


Saving .env to .env (4)
✅ Credenciais carregadas do .env


============================================
CÉLULA 5: Criar instância do Tradutor
============================================

In [ ]:
print("\n🚀 Inicializando tradutor...")
tradutor = TradutorGPT5Mini(
    api_key=API_KEY,
    api_version="2025-04-01-preview",
    azure_endpoint=AZURE_ENDPOINT,
    deployment_name=DEPLOYMENT_NAME
)
print("✅ Tradutor pronto para usar!")


🚀 Inicializando tradutor...
✅ Tradutor pronto para usar!


============================================
CÉLULA 6: Configurar parâmetros de tradução
============================================

In [ ]:
print("\n" + "="*60)
print("⚙️  CONFIGURAR TRADUÇÃO")
print("="*60)
URL = input("\n🌐 Digite a URL a traduzir: ").strip()
print("\n📚 Idiomas: português, inglês, espanhol, francês, alemão, italiano, japonês, chinês, etc.")
IDIOMA_ORIGEM = input("   Idioma de origem (padrão: português): ").strip().lower() or "português"
IDIOMA_DESTINO = input("   Idioma de destino (padrão: inglês): ").strip().lower() or "inglês"
ARQUIVO_SAIDA = f"traducao_{IDIOMA_DESTINO.replace(' ', '_')}.docx"
print(f"\n✅ Configuração:")
print(f"   URL: {URL}")
print(f"   Tradução: {IDIOMA_ORIGEM} → {IDIOMA_DESTINO}")
print(f"   Arquivo: {ARQUIVO_SAIDA}")


⚙️  CONFIGURAR TRADUÇÃO

🌐 Digite a URL a traduzir: https://en.wikipedia.org/wiki/Engineering

📚 Idiomas: português, inglês, espanhol, francês, alemão, italiano, japonês, chinês, etc.
   Idioma de origem (padrão: português): inglês
   Idioma de destino (padrão: inglês): português

✅ Configuração:
   URL: https://en.wikipedia.org/wiki/Engineering
   Tradução: inglês → português
   Arquivo: traducao_português.docx


============================================
CÉLULA 7: Executar tradução
============================================

In [ ]:
print("\n" + "="*60)
print("▶️  INICIANDO TRADUÇÃO...")
print("="*60)
try:
    arquivo = tradutor.processar_url_completo(
        url=URL,
        idioma_origem=IDIOMA_ORIGEM,
        idioma_destino=IDIOMA_DESTINO,
        caminho_saida=ARQUIVO_SAIDA
    )
    print(f"✅ SUCESSO! Arquivo: {arquivo}")
    sucesso = True
except Exception as e:
    print(f"❌ ERRO: {e}")
    sucesso = False


▶️  INICIANDO TRADUÇÃO...

🚀 Iniciando tradução com ChatGPT 5 Mini

🌐 Etapa 1: Extraindo texto...
   ✅ Título: Engineering - Wikipedia
   ✅ Caracteres: 50,029

🔄 Etapa 2: Traduzindo...
  🔄 Traduzindo chunk 1/7...
  🔄 Traduzindo chunk 2/7...
  🔄 Traduzindo chunk 3/7...
  🔄 Traduzindo chunk 4/7...
  🔄 Traduzindo chunk 5/7...
  🔄 Traduzindo chunk 6/7...
  🔄 Traduzindo chunk 7/7...
   ✅ Tradução concluída

💾 Etapa 3: Salvando arquivo...
   ✅ traducao_português.docx


✅ SUCESSO! Arquivo: traducao_português.docx


============================================
CÉLULA 8: Download do arquivo
============================================

In [ ]:
if sucesso:
    print("\n📥 Iniciando download do arquivo...")
    try:
        files.download(ARQUIVO_SAIDA)
        print("✅ Download iniciado! Verifique sua pasta de downloads.")
    except Exception as e:
        print(f"❌ Erro no download: {e}")
else:
    print("\n⚠️  Não foi possível fazer o download (erro na tradução)")


📥 Iniciando download do arquivo...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download iniciado! Verifique sua pasta de downloads.
